In [1]:
import numpy as np
import scipy.io as sio
import ordpy
from glob import glob
from pathlib import Path
from tqdm import tqdm

In [2]:
def mean_permutation_entropy(ts):
    channels, samples = ts.shape
    segment_length = 16*250
    segments = int(samples / segment_length)

    pe = []
    sc = []
    for channel in range(channels):
        for segment in range(segments):
            from_sample = segment*segment_length
            to_sample = (segment+1)*segment_length
            data = ts[channel, from_sample:to_sample]

            permutation_entropy, statistical_complexity = ordpy.complexity_entropy(
                    data, 
                    dx=3, 
                    dy=1, 
                    taux=1, 
                    tauy=1, 
                    probs=False, 
                    tie_precision=None
            )

            pe.append(permutation_entropy)
            sc.append(statistical_complexity)
        
    return np.mean(pe), np.mean(sc)

In [4]:
path = Path("/home/usuario/disco2/proyectos/interpolation/datasets/jaco/datos_128")
files = glob(f"{path}/Healthy/*.mat")

pe_list = []
sc_list = []
for file in tqdm(files):
    ts = sio.loadmat(file)["epochs_data"]
    ts = ts.reshape(128, -1)

    pe, sc = mean_permutation_entropy(ts)
    pe_list.append(pe)
    sc_list.append(sc)

sio.savemat("paris/pe_sc_healthy.mat", {"pe": pe_list, "sc": sc_list})

  0%|          | 0/32 [00:00<?, ?it/s]